In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random 
import torch
import torchvision
from torchvision import transforms, datasets
from torch.autograd import Variable
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import time

from ae_cifar10 import AutoEncoder
from utils import fgsm_attack, pgd_linf, to_numpy_array_cifar10, EarlyStopping
from vgg import VGG

c:\Users\malay\miniconda3\envs\cs776\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
attack_type = 'fgsm'
activation_type = 'gelu'
lr = 0.001
batch_size = 16
num_epochs = 10
dataset = 'cifar10'
# z_dim = 10
include_noise = True

# FGSM parameters
eps_fgsm = 5e-3
#PGD parameters
eps_pgd, alpha, num_iter = 5e-3, 1e-2, 5

In [3]:
# CIFAR10 Dataset
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_ae = transforms.Compose(
    [transforms.Resize((32, 32))])

transform_classifier = transforms.Compose(
    [transforms.Resize((224, 224))])

test_dataset = datasets.CIFAR10(root= './data', train = False, download =True, transform = transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Files already downloaded and verified


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

Selected device: cuda


In [5]:
classifier = VGG()
classifier.to(device)
classifier.load_state_dict(torch.load('./models/vgg16.pth.tar')['vgg16_state_dict'])

<All keys matched successfully>

## Test Evaluation on our proposed architecture

In [21]:
include_noise = True

if include_noise:
    file_path = f'./models/ae_cifar10_{attack_type}_{activation_type}_proposed.pth.tar'
else:
    file_path = f'./models/ae_cifar10_{attack_type}_{activation_type}_vanilla.pth.tar'

convAE = AutoEncoder(device, include_noise)
convAE.to(device)
convAE.load_state_dict(torch.load(file_path)['convAE_state_dict'])

<All keys matched successfully>

In [22]:
correct, total = 0, 0

for _, (imgs, labels) in enumerate(test_loader):
    batch_size = imgs.shape[0]
    imgs, labels = Variable(imgs.to(device), requires_grad=True), Variable(labels.to(device))
    
    if attack_type == 'fgsm':
        adv_imgs, _ = fgsm_attack(classifier, imgs, labels, eps_fgsm, dataset)
    else:
        adv_imgs, _ = pgd_linf(classifier, imgs, labels, eps_pgd, alpha, num_iter)

    adv_imgs = adv_imgs.to(device)
    adv_imgs_ = transform_ae(adv_imgs).to(device)

    with torch.no_grad():
        rec_imgs = convAE(adv_imgs_)
        rec_imgs = transform_classifier(rec_imgs)
        
    y_preds = classifier(rec_imgs).argmax(dim=1)
    correct += (y_preds==labels).sum().item()
    total += labels.size(0)

In [23]:
print("Epsilon: {}, Test Accuracy: {}".format(eps_fgsm, correct / total))
print(f'{correct} {total}')

Epsilon: 0.005, Test Accuracy: 0.5716
5716 10000


## Test Evaluation on Vanilla Autoencoder

In [9]:
include_noise = False

if include_noise:
    file_path = f'./models/ae_cifar10_{attack_type}_{activation_type}_proposed.pth.tar'
else:
    file_path = f'./models/ae_cifar10_{attack_type}_{activation_type}_vanilla.pth.tar'

convAE = AutoEncoder(device, include_noise)
convAE.to(device)
convAE.load_state_dict(torch.load(file_path)['convAE_state_dict'])

<All keys matched successfully>

In [10]:
correct, total = 0, 0

for _, (imgs, labels) in enumerate(test_loader):
    batch_size = imgs.shape[0]
    imgs, labels = Variable(imgs.to(device), requires_grad=True), Variable(labels.to(device))
    
    if attack_type == 'fgsm':
        adv_imgs, _ = fgsm_attack(classifier, imgs, labels, eps_fgsm, dataset)
    else:
        adv_imgs, _ = pgd_linf(classifier, imgs, labels, eps_pgd, alpha, num_iter)

    adv_imgs = adv_imgs.to(device)
    adv_imgs_ = transform_ae(adv_imgs).to(device)

    with torch.no_grad():
        rec_imgs = convAE(adv_imgs_)
        rec_imgs = transform_classifier(rec_imgs)
        
    y_preds = classifier(rec_imgs).argmax(dim=1)
    correct += (y_preds==labels).sum().item()
    total += labels.size(0)

In [11]:
print("Epsilon: {}, Test Accuracy: {}".format(eps_fgsm, correct / total))

Epsilon: 0.005, Test Accuracy: 0.5654


## Inference latency

In [12]:
tot_time = 0

for _, (imgs, labels) in enumerate(test_loader):
    batch_size = imgs.shape[0]
    imgs, labels = Variable(imgs.to(device), requires_grad=True), Variable(labels.to(device))

    if attack_type == 'fgsm':
        adv_imgs, _ = fgsm_attack(classifier, imgs, labels, eps_fgsm, dataset)
    else:
        adv_imgs, _ = pgd_linf(classifier, imgs, labels, eps_pgd, alpha, num_iter)

    attack_comp = time.time()

    adv_imgs = adv_imgs.to(device)
    adv_imgs_ = transform_ae(adv_imgs).to(device)

    with torch.no_grad():
        rec_imgs = convAE(adv_imgs_)
        rec_imgs = transform_classifier(rec_imgs)
        
    y_preds = classifier(rec_imgs).argmax(dim=1)
    tot_time += time.time() - attack_comp

In [14]:
print(f'Time taken for defense against a single instance of attack: {(tot_time)/len(test_dataset):.4f} sec')
print(f'Total time taken: {tot_time//60:.0f} min {tot_time%60:.2f} sec')

Time taken for defense against a single instance of attack: 0.0004 sec
Total time taken: 0 min 4.38 sec
